In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fixation = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fixation.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fixation = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fixation.csv')
data_trial_choice = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv')

data_prolific = pd.read_csv(
    r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv'
)
data_prolific = data_prolific.rename(columns={'participant_id': 'prolificID'})

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fixation)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fixation)],
        [len(data_trial_choice)],
        [len(data_subject)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fixation', 'data_et_choice', 
           'data_trial', 'data_trial_fixation', 'data_trial_choice', 
           'data_subject', 'data_prolific'])
print(overview)

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,56,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                     length
data_et              263965
data_et_fixation      78778
data_et_choice        35125
data_trial            10476
data_trial_fixation    1468
data_trial_choice      1503
data_subject             21
data_prolific            36


# Finished experiment? 

In [3]:
if 'trial_length' in data_subject.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [4]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        (data_et['task_nr']!=3), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        (data_et['task_nr']!=3), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_trial.loc[
        (data_trial['run_id']==subject) &
        (data_trial['trial_type']=='eyetracking-fix-object') &
        (data_trial['task_nr']!=3), 
        'count'
    ].mean()

    m_count_choice = data_trial.loc[
        (data_trial['run_id']==subject) &
        (data_trial['trial_type']=='eyetracking-choice'), 
        'count'
    ].mean()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_subject = data_subject.merge(output, on='run_id', how='left')

In [5]:
data_subject = data_subject.merge(data_prolific.loc[:, ['prolificID', 'status']], on='prolificID', how='left')

In [6]:
data_subject.loc[data_subject['status']=='AWAITING REVIEW', 
                 ['run_id', 'prolificID', 'status', 'trial_length',
                 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
                 'm_choseTop']
                ]

,run_id,prolificID,status,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [7]:
if 'choice_rt' in data_subject.columns:
    print('choice_rt already added!')
else: 
    grouped = data_trial.loc[data_trial['trial_type']=='eyetracking-choice', :] \
        .groupby(['prolificID'])['trial_duration_exact'].mean() \
        .reset_index() \
        .rename(columns={'trial_duration_exact': 'choice_rt'})
    data_subject = data_subject.merge(grouped, on='prolificID', how='left')

In [8]:
data_subject.loc[:, ['run_id', 'prolificID', 'webcam_fps', 'fps', 'choice_rt']]

,run_id,prolificID,webcam_fps,fps,choice_rt
0,1,5fccc8ac636416a4288a9f3d,30.000000,2.030567,3812.6625
1,11,5c5684ef9d244c0001b29f1e,30.000000,21.456080,1659.6250
2,12,5b8969006651ea000118e42e,30.000000,17.997210,1909.5875
3,13,5b4e65de09f5af0001080f45,30.000000,21.704953,2041.9750
4,19,5f860a189bc695131f22ec9c,30.000000,21.565793,2022.9000
5,2,Tim2,30.000000,21.368029,NaN
6,24,5f77a902fc647a327b77004a,30.000000,15.268646,2682.6250
7,25,5cf0ccf488abf10019fb775d,30.000000,13.329251,3321.0375
8,28,5f561a95aa1c4ea13672f138,30.000000,24.590346,1334.3750
9,30,5eeaa0ffaa6af11cf32ce057,29.970030,14.284995,1649.6875


# Bonus payment

In [9]:
data_bonus = data_subject.loc[:, ['run_id', 'prolificID', 'chosenAmount', 'chosenDelay']] \
    .sort_values(by='run_id')

data_bonus = data_prolific \
    .merge(data_bonus, on='prolificID', how='left')
data_bonus = data_bonus.loc[data_bonus['status']=='APPROVED', :]

data_bonus.loc[: , ['chosenAmount', 'chosenDelay']] = \
    data_bonus.loc[:, ['chosenAmount', 'chosenDelay']] \
    .replace(['Today', 'Tomorrow', '7 days', '15 days', '30 days', '90 days', '180 days'], 
             [0, 1, 7, 15, 30, 90, 180]) \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) \
    .astype(float)

In [10]:
# For two subjects, the bonus payment was not recorded
data_bonus.loc[
    data_bonus['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', '5b8969006651ea000118e42e']
    ), ['chosenAmount', 'chosenDelay']] = [5, 1]

data_bonus.loc[data_bonus[['chosenAmount', 'chosenDelay']].isnull().any(axis=1), :]

,session_id,prolificID,status,started_datetime,completed_date_time,time_taken,age,num_approvals,num_rejections,prolific_score,...,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Student Status,run_id,chosenAmount,chosenDelay


In [11]:
data_bonus.columns

Index(['session_id', 'prolificID', 'status', 'started_datetime',
       'completed_date_time', 'time_taken', 'age', 'num_approvals',
       'num_rejections', 'prolific_score', 'reviewed_at_datetime',
       'entered_code', 'Country of Birth', 'Current Country of Residence',
       'Employment Status', 'First Language', 'Nationality', 'Sex',
       'Student Status', 'run_id', 'chosenAmount', 'chosenDelay'],
      dtype='object')

In [20]:
data_bonus['date']=data_bonus.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_bonus['reward_pound'] = data_bonus['chosenAmount'] * 0.75
data_bonus['due_on'] = data_bonus['date'] +  data_bonus['chosenDelay'].map(datetime.timedelta) 
data_bonus.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'date', 'chosenAmount', 'chosenDelay', 
                   'reward_pound', 'due_on'
               ]
              ]. sort_values(by='due_on')

,prolificID,run_id,Nationality,Sex,date,chosenAmount,chosenDelay,reward_pound,due_on
20,5f77a902fc647a327b77004a,24.0,United Kingdom,Female,2021-01-18,3.0,0.0,2.250,2021-01-18
33,5f8472685956c40c720d0936,36.0,Italy,Male,2021-01-18,0.5,0.0,0.375,2021-01-18
29,5f561a95aa1c4ea13672f138,28.0,Italy,Male,2021-01-18,2.5,0.0,1.875,2021-01-18
27,5eeaa0ffaa6af11cf32ce057,30.0,Poland,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
34,5f0cb319d29147695796a208,37.0,Italy,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
14,5c5684ef9d244c0001b29f1e,11.0,United States,Male,2021-01-18,3.5,0.0,2.625,2021-01-18
25,5fcbb3066829ff1a601fe080,42.0,United Kingdom,Female,2021-01-18,5.0,1.0,3.750,2021-01-19
11,5fea6632bf9ae4a79153efdf,9.0,Serbia,Male,2021-01-18,5.0,1.0,3.750,2021-01-19
15,5b8969006651ea000118e42e,12.0,Italy,Female,2021-01-18,5.0,1.0,3.750,2021-01-19
0,5f860a189bc695131f22ec9c,19.0,Poland,Male,2021-01-18,4.5,7.0,3.375,2021-01-25


In [31]:
bonus_due_today = data_bonus.loc[data_bonus['due_on']==datetime.datetime.now().date(), ['prolificID', 'reward_pound']]
bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)

bonus_due_today.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/bonusToday.csv', index=False, header=False)